# Anna.AI

## Motivation:

Hiện nay đã có các hệ thống chatbot NLP rất tốt (ChatGPT, Siri, Alexa), nhưng chưa có 1 visual người ảo.
Tức đã có input (audio/text) sang output (text). Nhưng chưa có từ text sang 1 visual.
Để xây dựng 1 virtual human assitant mà tương tác được cần sinh ra Output:
- Hình ảnh: text to visual: 3D scan da người, các texture, mesh và quan trọng nhất là cử chỉ của người đó.
- Âm thanh: text to speech.

Với da người texture và mesh đã có R3DS Wrap4D(russian3dscanner.com) hay iClone7, có rất nhiều 3d scan texture được bán sẵn (3dscanstore.com).
Với cử chỉ tương ứng với từng đoạn text thì gồm có cử chỉ của khuôn mặt (đã có Omniverse Audio2Face) và cử chỉ dáng người.
Việc tạo ra cử chỉ dáng người (Gesture Generation) phần lớn sẽ được tạo ra manual bằng Move.ai (quay bằng iPhone) để cache lại và chỉ cần inferece (sinh) cho những câu text phức tạp không có pattern sẵn.
Việc sinh cử chỉ dáng người (Gesture Generation) có thể từ text hoặc bằng audio speech. Tuy nhiên text2gesture thì rất khó maching do không có đủ thông tin,
audio thì có được các thông tin về từng khoảng thời gian, sự lên xuống nên phù hợp hơn.
Vì vậy mục tiêu của em là xây dựng model AI để sinh gesture (cử chỉ dáng người) từ input đầu vào là audio speech.

## Trang paper chính tham khảo:
- Source code: https://pantomatrix.github.io/BEAT/
- Cuộc thi GENEA CHALLENGE 2022 :
https://paperswithcode.com/sota/gesture-generation-on-beat?p=beat-a-large-scale-semantic-and-emotional


## Problem statements:

* Input:
Audio Speech: Âm thanh của các bài TED Talk

* Output:
Gesture Generation: 3D keypoint (tập các điểm tương ứng với vị trí của tay, chân, mắt...) tương ứng với giọng nói

* Dataset:
BEAT: Body-Expression-Audio-Text Dataset
(https://github.com/PantoMatrix/BEAT)

## Các phưng pháp liên quan:
- Paperwithcode : https://paperswithcode.com/.../gesture-generation-on-beat...

## Demo

Em đã load được model người trên website:
https://hmthanh.github.io/3d-human-model/

Kaggle đang thực hiện: https://www.kaggle.com/hmthanh/anna-ai/

Github: https://github.com/hmthanh/BEAT

## Audio-Driven Gesture Generation for Virtual Human Assistant
(Xây dựng hệ thống sinh cử chỉ dựa trên Âm thanh)

processing dataset


In [1]:
!git clone https://github.com/hmthanh/youtube-gesture-dataset

Cloning into 'youtube-gesture-dataset'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 68 (delta 4), reused 4 (delta 0), pack-reused 52
Unpacking objects: 100% (68/68), 39.33 KiB | 1.27 MiB/s, done.


### 0. Prepare

In [11]:
!git clone https://github.com/hmthanh/BEAT.git
!mv ./BEAT/* ./

Cloning into 'BEAT'...
remote: Enumerating objects: 478, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 478 (delta 68), reused 35 (delta 13), pack-reused 341
Receiving objects: 100% (478/478), 253.37 MiB | 28.90 MiB/s, done.
Resolving deltas: 100% (124/124), done.


In [12]:
!pip install llvmlite --ignore-installed
!pip install -r requirements.txt -q

# !rm -rf BEAT

ERROR: Cannot uninstall 'llvmlite'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 20.5 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.55.2 requires llvmlite<0.39,>=0.38.0rc1, but you have llvmlite 0.39.1 which is incompatible.


In [2]:
# Clone the repo
!mkdir codes 
!mkdir datasets
!mkdir datasets/beat_cache/
!mkdir outputs
!mkdir outputs/audio2pose
!mkdir outputs/audio2pose/custom/
!mkdir outputs/audio2pose/wandb/
# !cd codes/ && git clone https://github.com/PantoMatrix/BEAT.git 

### Download cache


In [ ]:
# download cache
!pip install -U --no-cache-dir gdown --pre
!cd datasets/beat_cache/ && gdown 18fy5NZyw5hMqWSG261RwXkJbyrWnE__3
!cd datasets/beat_cache/ && unzip beat_4english_15_141.zip

In [9]:
!mkdir /kaggle/working/datasets
!cp /kaggle/input/audio-driven-gesture-generation-dataset/audio-driven-gesture-generation-dataset/* /kaggle/working/datasets

mkdir: cannot create directory ‘/kaggle/working/datasets’: File exists
cp: -r not specified; omitting directory '/kaggle/input/audio-driven-gesture-generation-dataset/audio-driven-gesture-generation-dataset/test'
cp: -r not specified; omitting directory '/kaggle/input/audio-driven-gesture-generation-dataset/audio-driven-gesture-generation-dataset/train'
cp: -r not specified; omitting directory '/kaggle/input/audio-driven-gesture-generation-dataset/audio-driven-gesture-generation-dataset/weights'


In [14]:
!ls /kaggle/working/datasets

beat_cache  vocab.pkl


In [3]:
# !pip install -r requirements.txt --quiet
!pip install configargparse

## Test

In [20]:
!python test.py -c ./configs/camn_beat_4english_15_141.yaml --new_cache False

/opt/conda/lib/python3.7/site-packages/llvmlite/llvmpy/__init__.py:4: UserWarning: The module `llvmlite.llvmpy` is deprecated and will be removed in the future.
  "The module `llvmlite.llvmpy` is deprecated and will be removed in the "
/opt/conda/lib/python3.7/site-packages/llvmlite/llvmpy/core.py:9: UserWarning: The module `llvmlite.llvmpy.core` is deprecated and will be removed in the future. Equivalent functionality is provided by `llvmlite.ir`.
  "The module `llvmlite.llvmpy.core` is deprecated and will be removed in "
/opt/conda/lib/python3.7/site-packages/llvmlite/llvmpy/passes.py:18: UserWarning: The module `llvmlite.llvmpy.passes` is deprecated and will be removed in the future. If you are using this code, it should be inlined into your own project.
  "The module `llvmlite.llvmpy.passes` is deprecated and will be removed in "
Traceback (most recent call last):
  File "test.py", line 25, in <module>
    from dataloaders import data_tools
  File "/kaggle/working/dataloaders/data_

In [21]:
!ls /datasets/beat_cache/beat_4english_15_141

ls: cannot access '/datasets/beat_cache/beat_4english_15_141': No such file or directory


In [ ]:
/outputs/audio2pose/custom/exp_name/9999/xxx.bvh

In [4]:
!find datasets

find: ‘datasets’: No such file or directory


In [1]:
!find /kaggle/input/audio-driven-gesture-generation-dataset/audio-driven-gesture-generation-dataset

find: ‘/kaggle/input/audio-driven-gesture-generation-dataset/audio-driven-gesture-generation-dataset’: No such file or directory


In [2]:
!ls

__notebook_source__.ipynb


In [5]:
!ls

__notebook_source__.ipynb


In [ ]:
audio-driven-gesture-generation-dataset

In [1]:
!ls

__notebook_source__.ipynb


Các link đáng chú ý:

https://www.youtube.com/watch?v=-TS2iLhYP28

## Inference

In [1]:
# Clone the repo
!mkdir codes 
!mkdir datasets
!mkdir datasets/beat_cache/
!mkdir outputs
!mkdir outputs/audio2pose
!mkdir outputs/audio2pose/custom/s
!mkdir outputs/audio2pose/wandb/

In [2]:
!cd codes/ && git clone https://github.com/hmthanh/BEAT.git 

Cloning into 'BEAT'...
remote: Enumerating objects: 491, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 491 (delta 76), reused 49 (delta 21), pack-reused 341
Receiving objects: 100% (491/491), 253.37 MiB | 38.27 MiB/s, done.
Resolving deltas: 100% (132/132), done.


In [3]:
# download cache
!pip install -U --no-cache-dir gdown --pre
!cd datasets/beat_cache/ && gdown 18fy5NZyw5hMqWSG261RwXkJbyrWnE__3
!cd datasets/beat_cache/ && unzip beat_4english_15_141.zip

Downloading...
From: https://drive.google.com/uc?id=18fy5NZyw5hMqWSG261RwXkJbyrWnE__3
To: /kaggle/working/datasets/beat_cache/beat_4english_15_141.zip
100%|█████████████████████████████████████████| 899M/899M [00:03<00:00, 230MB/s]
Archive:  beat_4english_15_141.zip
   creating: beat_4english_15_141/
   creating: beat_4english_15_141/test/
   creating: beat_4english_15_141/test/bvh_full/
  inflating: beat_4english_15_141/test/bvh_full/2_scott_0_103_103.bvh  
  inflating: beat_4english_15_141/test/bvh_full/2_scott_0_111_111.bvh  
  inflating: beat_4english_15_141/test/bvh_full/2_scott_0_1_1.bvh  
  inflating: beat_4english_15_141/test/bvh_full/2_scott_0_2_2.bvh  
  inflating: beat_4english_15_141/test/bvh_full/2_scott_0_3_3.bvh  
  inflating: beat_4english_15_141/test/bvh_full/2_scott_0_4_4.bvh  
  inflating: beat_4english_15_141/test/bvh_full/2_scott_0_5_5.bvh  
  inflating: beat_4english_15_141/test/bvh_full/2_scott_0_65_65.bvh  
  inflating: beat_4english_15_141/test/bvh_full/2_scott

## Start inference here


In [6]:
!cp -R /kaggle/input/audio-driven-gesture-generation-dataset/* /kaggle/working

In [16]:
# !git clone https://github.com/hmthanh/BEAT.git
    
!mv /kaggle/working/BEAT/* /kaggle/working/

mv: cannot stat '/kaggle/working/BEAT/*': No such file or directory


In [23]:
!wget https://raw.githubusercontent.com/hmthanh/BEAT/main/requirements.txt

--2023-02-06 13:34:13--  https://raw.githubusercontent.com/hmthanh/BEAT/main/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 583 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     583  --.-KB/s    in 0s      

2023-02-06 13:34:14 (16.1 MB/s) - ‘requirements.txt’ saved [583/583]



In [24]:
!pip install -r /kaggle/working/requirements.txt --quiet

ERROR: Cannot uninstall 'llvmlite'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [21]:
import os
import signal
import time
import csv
import sys
import warnings
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.multiprocessing as mp
import numpy as np
import time
import pprint
from loguru import logger
import wandb

from utils import config, logger_tools, other_tools, metric
from dataloaders import data_tools
from dataloaders.build_vocab import Vocab
from optimizers.optim_factory import create_optimizer
from optimizers.scheduler_factory import create_scheduler
from optimizers.loss_factory import get_loss_func


ModuleNotFoundError: No module named 'loguru'